In [1]:
import calendar
import pandas as pd
import panel as pn

from datetime import date, timedelta
from sqlalchemy import create_engine

pn.extension('tabulator')

engine = create_engine("sqlite:///c:\\ruby\\port_lite\\db\\development.sqlite3")
conlite = engine.connect()

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"
one_path = "\\Users\\User\\OneDrive\\Documents\\Data\\"

today = date.today()
today

datetime.date(2022, 8, 7)

In [2]:
cols = 'trade name qty price active reason market xdate'.split()
colt = 'trade name qty target active reason market xdate'.split()

In [3]:
pd.read_sql_query('SELECT * FROM orders ORDER BY name', conlite)

,id,trade,name,qty,price,active,reason,market,xdate
0,4,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,5,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
2,6,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
3,16,S,BCH,3000,21.00,0,CP1S,SET100,2022-08-31
4,7,B,BCPG,9000,10.00,0,52WL,SET100,2022-02-02
5,8,B,DELTA,300,450.00,0,52WL,SET,2022-02-02
6,14,S,DIF,10000,14.60,0,CUT,SET,2022-08-16
7,17,S,DOHOME,2800,20.30,0,RD20pct,SET100,2022-02-02
8,9,B,IVL,3000,43.50,0,13WL,SET50,2022-05-31
9,1,B,MCS,5000,10.60,0,52WL,SET,2022-09-03


### After call Ord-Log.bat which calculates spreads between target price and SET price.

In [4]:
file_name   = 'orders-log.csv'
input_file = data_path + file_name
#col_names   = ['trade','name','spreads','reason','market','qty','target_price','set_price','chg_amt','chg_pct','active']
df_log = pd.read_csv(input_file, sep=',',  index_col=None)
df_log

,trade,name,spd,reason,market,qty,target,current,change,percent,active,xdate
0,B,AP,-14,MinP,SET100,10000,9.50,10.40,0.20,+1.96%,0,2022-05-10
1,B,ASK,-31,52WL,SET,3000,30.00,37.75,1.25,+3.42%,0,2022-03-03
2,B,BANPU,-13,1L,SET50,12000,11.50,12.80,0.00,0.00%,0,2022-04-11
3,B,BCPG,-3,52WL,SET100,9000,10.00,10.30,-0.10,-0.96%,0,2022-02-02
4,B,DELTA,-45,52WL,SET,300,450.00,540.00,-2.00,-0.37%,0,2022-02-02
5,B,IVL,2,13WL,SET50,3000,43.50,43.00,-1.00,-2.27%,0,2022-05-31
6,B,MCS,-7,52WL,SET,5000,10.60,11.30,0.00,0.00%,0,2022-09-03
7,B,PTTEP,-10,RD05pct,SET50,600,146.00,151.00,-3.50,-2.27%,0,2022-08-15
8,B,RBF,-18,52WL,SET100,9000,11.40,13.20,0.00,0.00%,0,2022-02-02
9,B,SINGER,-42,52WL,SET100,3000,35.50,46.00,1.25,+2.79%,0,2022-02-02


In [5]:
def calc(vals):
    spreads = vals
    if abs(spreads) < 10:
        return 1
    else:
        return 0

In [6]:
df_log['active'] = df_log['spd'].apply(calc)

In [7]:
df_log.groupby('active')['trade'].count()

active
0    12
1     6
Name: trade, dtype: int64

In [8]:
sql = '''
DELETE FROM orders'''
rp = conlite.execute(sql)
rp.rowcount

18

In [9]:
final = df_log[colt]
final

,trade,name,qty,target,active,reason,market,xdate
0,B,AP,10000,9.50,0,MinP,SET100,2022-05-10
1,B,ASK,3000,30.00,0,52WL,SET,2022-03-03
2,B,BANPU,12000,11.50,0,1L,SET50,2022-04-11
3,B,BCPG,9000,10.00,1,52WL,SET100,2022-02-02
4,B,DELTA,300,450.00,0,52WL,SET,2022-02-02
5,B,IVL,3000,43.50,1,13WL,SET50,2022-05-31
6,B,MCS,5000,10.60,1,52WL,SET,2022-09-03
7,B,PTTEP,600,146.00,0,RD05pct,SET50,2022-08-15
8,B,RBF,9000,11.40,0,52WL,SET100,2022-02-02
9,B,SINGER,3000,35.50,0,52WL,SET100,2022-02-02


In [10]:
rcds = final.values.tolist()
rcds

[['B', 'AP', 10000, 9.5, 0, 'MinP', 'SET100', '2022-05-10'],
 ['B', 'ASK', 3000, 30.0, 0, '52WL', 'SET', '2022-03-03'],
 ['B', 'BANPU', 12000, 11.5, 0, '1L', 'SET50', '2022-04-11'],
 ['B', 'BCPG', 9000, 10.0, 1, '52WL', 'SET100', '2022-02-02'],
 ['B', 'DELTA', 300, 450.0, 0, '52WL', 'SET', '2022-02-02'],
 ['B', 'IVL', 3000, 43.5, 1, '13WL', 'SET50', '2022-05-31'],
 ['B', 'MCS', 5000, 10.6, 1, '52WL', 'SET', '2022-09-03'],
 ['B', 'PTTEP', 600, 146.0, 0, 'RD05pct', 'SET50', '2022-08-15'],
 ['B', 'RBF', 9000, 11.4, 0, '52WL', 'SET100', '2022-02-02'],
 ['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', '2022-02-02'],
 ['B', 'SSP', 10000, 8.75, 0, '52WL', 'SET', '2022-02-02'],
 ['B', 'TMT', 12000, 7.9, 1, 'RD20pct', 'SET', '2022-08-27'],
 ['B', 'TTB', 90000, 0.95, 0, '52WL', 'SET50', '2022-02-02'],
 ['S', 'BCH', 3000, 21.0, 1, 'CP1S', 'SET100', '2022-08-31'],
 ['S', 'DIF', 10000, 14.6, 1, 'CUT', 'SET', '2022-08-16'],
 ['S', 'DOHOME', 2800, 20.3, 0, 'RD20pct', 'SET100', '2022-02-02'],
 ['S', '

In [11]:
for rcd in rcds:
    print(rcd)

['B', 'AP', 10000, 9.5, 0, 'MinP', 'SET100', '2022-05-10']
['B', 'ASK', 3000, 30.0, 0, '52WL', 'SET', '2022-03-03']
['B', 'BANPU', 12000, 11.5, 0, '1L', 'SET50', '2022-04-11']
['B', 'BCPG', 9000, 10.0, 1, '52WL', 'SET100', '2022-02-02']
['B', 'DELTA', 300, 450.0, 0, '52WL', 'SET', '2022-02-02']
['B', 'IVL', 3000, 43.5, 1, '13WL', 'SET50', '2022-05-31']
['B', 'MCS', 5000, 10.6, 1, '52WL', 'SET', '2022-09-03']
['B', 'PTTEP', 600, 146.0, 0, 'RD05pct', 'SET50', '2022-08-15']
['B', 'RBF', 9000, 11.4, 0, '52WL', 'SET100', '2022-02-02']
['B', 'SINGER', 3000, 35.5, 0, '52WL', 'SET100', '2022-02-02']
['B', 'SSP', 10000, 8.75, 0, '52WL', 'SET', '2022-02-02']
['B', 'TMT', 12000, 7.9, 1, 'RD20pct', 'SET', '2022-08-27']
['B', 'TTB', 90000, 0.95, 0, '52WL', 'SET50', '2022-02-02']
['S', 'BCH', 3000, 21.0, 1, 'CP1S', 'SET100', '2022-08-31']
['S', 'DIF', 10000, 14.6, 1, 'CUT', 'SET', '2022-08-16']
['S', 'DOHOME', 2800, 20.3, 0, 'RD20pct', 'SET100', '2022-02-02']
['S', 'RCL', 3000, 46.0, 0, 'RD20pct', '

In [12]:
sql = """
INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
"""
print(sql)


INSERT INTO orders (trade, name, qty, price, active, reason, market, xdate)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)



In [13]:
for rcd in rcds:
    conlite.execute(sql, rcd)

In [14]:
sql = '''
SELECT *
FROM orders
ORDER BY trade, name'''
df_orders = pd.read_sql(sql, conlite)
df_tab = pn.widgets.Tabulator(df_orders, layout='fit_data', width=800)
df_tab

Tabulator(layout='fit_data', value=    id trade  ..., width=800)

In [15]:
file_name = 'orders.csv'
data_file = data_path + file_name
output_file = csv_path + file_name
box_file = box_path + file_name
one_file = one_path + file_name

df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(output_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(data_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(box_file, header=True, index=False)
df_orders[cols].sort_values(['trade','name'],ascending=[True,True]).to_csv(one_file, header=True, index=False)